In [3]:
#The purpose of this code is to process audio files in a specified directory, 
#segmenting each audio file into shorter audio clips of a predetermined duration, 
#and then save these segmented audio clips into a different directory.

import os
import librosa
import soundfile as sf

def segment_audio(audio_file, segment_duration=40): #The segment_audio function takes an audio file path and an optional segment_duration parameter, with a default value of 40 seconds.
    try:
        # Load audio file
        audio, sr = librosa.load(audio_file, sr=None, mono=True)
        
        # Calculate the total number of segments
        total_segments = len(audio) // (sr * segment_duration)
        
        segments = []
        # Segment the audio file
        for i in range(total_segments):
            start_sample = i * sr * segment_duration
            end_sample = start_sample + sr * segment_duration
            segment = audio[start_sample:end_sample]
            segments.append(segment)
        
        return segments, sr
    except Exception as e:
        print(f"Error segmenting {audio_file}: {e}")
        return [], None

# Directory containing the audio files
data_directory = "/Users/chamudi/Desktop/Dataset"

# Directory to save segmented audio files
segmented_data_directory = "/Users/chamudi/Desktop/Segmanted_Data_Model_10"
os.makedirs(segmented_data_directory, exist_ok=True)

# Process each audio file
for file in os.listdir(data_directory):
    if file.endswith(".mp3"):
        audio_path = os.path.join(data_directory, file)
        segments, sr = segment_audio(audio_path)
        if segments and sr:
            # Save segmented audio files
            for i, segment in enumerate(segments):
                segment_file = f"{os.path.splitext(file)[0]}_segment_{i}.wav"
                segment_path = os.path.join(segmented_data_directory, segment_file)
                sf.write(segment_path, segment, sr)


In [4]:
import os
import librosa
import soundfile as sf
import numpy as np

def apply_stft(segment, sr, n_fft=2048, hop_length=512):
    stft_result = librosa.stft(segment, n_fft=n_fft, hop_length=hop_length)
    stft_magnitude = np.abs(stft_result)
    stft_magnitude_db = librosa.amplitude_to_db(stft_magnitude, ref=np.max)
    return stft_magnitude_db

# Directory containing the audio files
data_directory = "/Users/chamudi/Desktop/Dataset"

# Directory to save features or processed data
features_directory = "/Users/chamudi/Desktop/Features_Data_Model_10"
os.makedirs(features_directory, exist_ok=True)

# Process each audio file
for file in os.listdir(data_directory):
    if file.endswith(".mp3"):
        audio_path = os.path.join(data_directory, file)
        segments, sr = segment_audio(audio_path, segment_duration=40)
        if segments and sr:
            # Process and save features for each segment
            for i, segment in enumerate(segments):
                stft_features = apply_stft(segment, sr)
                # Example feature processing or saving
                feature_file = f"{os.path.splitext(file)[0]}_segment_{i}_features.npy"
                feature_path = os.path.join(features_directory, feature_file)
                np.save(feature_path, stft_features)

print("Feature extraction complete.")

Feature extraction complete.


In [6]:
import os
import numpy as np
import librosa

def load_stft_features(feature_path):
    """
    Load STFT features from a .npy file.
    """
    return np.load(feature_path)

def find_peaks(stft_features, sr, n_fft, hop_length, pre_max, post_max, pre_avg, post_avg, delta, wait):
    """
    Identify peaks in the STFT features using librosa's utilities.
    """
    # Assuming stft_features is a 2D STFT matrix, you might first want to aggregate it into a 1D time series
    # Here, we calculate the spectral flux as an example of such aggregation
    spectral_flux = librosa.onset.onset_strength(S=stft_features, sr=sr)

    # Use librosa's peak picking with keyword arguments
    peaks = librosa.util.peak_pick(x=spectral_flux, pre_max=pre_max, post_max=post_max,
                                   pre_avg=pre_avg, post_avg=post_avg, delta=delta, wait=wait)
    return peaks

# Directory where STFT features are stored
features_directory = "/Users/chamudi/Desktop/Features_Data_Model_10"

# Directory to save the fingerprints
fingerprints_directory = "/Users/chamudi/Desktop/Fingerprints_Data_Model_10"
os.makedirs(fingerprints_directory, exist_ok=True)

# STFT parameters (should match those used during feature extraction)
sr = 22050  # Sample rate
n_fft = 2048
hop_length = 512

# Peak picking parameters
pre_max = 1
post_max = 1
pre_avg = 3
post_avg = 3
delta = 0.2
wait = 0

# Process each STFT feature file
for feature_file in os.listdir(features_directory):
    if feature_file.endswith(".npy"):
        feature_path = os.path.join(features_directory, feature_file)
        stft_features = load_stft_features(feature_path)
        
        # Extract peaks as fingerprints
        peaks = find_peaks(stft_features, sr, n_fft, hop_length, pre_max, post_max, pre_avg, post_avg, delta, wait)
        
        # Save fingerprints
        fingerprint_file = feature_file.replace("_features.npy", "_fingerprints.npy")
        fingerprint_path = os.path.join(fingerprints_directory, fingerprint_file)
        np.save(fingerprint_path, peaks)

print("Fingerprint generation complete.")

Fingerprint generation complete.
